In [2]:
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt


In [ ]:
df = pd.read_pickle('./data_final.pkl') #importing cleaned data from the other file

In [4]:
station_flow_annual = df.groupby(by = 'Station')['Net_Flow'].sum()
station_flow_annual.sort_values(ascending = False,inplace = True)
print(station_flow_annual.index[:20]) #list of top 20

Index(['34 ST-PENN STA', 'GRD CNTRL-42 ST', '34 ST-HERALD SQ', '23 ST',
       'TIMES SQ-42 ST', '14 ST-UNION SQ', '42 ST-PORT AUTH', 'FULTON ST',
       '86 ST', '125 ST', 'CANAL ST', '59 ST COLUMBUS', '59 ST', '14 ST',
       '47-50 STS ROCK', '96 ST', 'FLUSHING-MAIN', 'CHAMBERS ST',
       'PATH NEW WTC', '72 ST'],
      dtype='object', name='Station')


In [8]:
df_location = pd.read_csv('~/Documents/Github/Strategic_placements_of_resources/Strategic_placements_of_resources/data/external/StationEntrances.csv')

In [11]:
df_location.head()

,Division,Line,Station_Name,Station_Latitude,Station_Longitude,Route_1,Route_2,Route_3,Route_4,Route_5,...,Staffing,Staff_Hours,ADA,ADA_Notes,Free_Crossover,North_South_Street,East_West_Street,Corner,Latitude,Longitude
0,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NW,40.775149,-73.912074
1,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NE,40.774810,-73.912151
2,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NE,40.775025,-73.911891
3,BMT,Astoria,Ditmars Blvd,40.775036,-73.912034,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,23rd Ave,NW,40.774938,-73.912337
4,BMT,Astoria,Astoria Blvd-Hoyt Av,40.770258,-73.917843,N,Q,NaN,NaN,NaN,...,FULL,NaN,False,NaN,True,31st St,Hoyt Ave South,SW,40.770313,-73.917978


In [13]:
location = pd.DataFrame(
                    df_location.groupby(by = 'Station_Name')[['Station_Latitude','Station_Longitude']].mean())

,Station_Latitude,Station_Longitude
Station_Name,,
103rd St,40.781165,-73.929092
104th St-102nd St,40.695178,-73.844330
104th St-Oxford Av,40.681711,-73.837683
110th St,40.795020,-73.944250
110th St-Central Park North,40.799075,-73.951822
